# Use function calling and grounding with Google Search to create a research tool

[link](https://partner.cloudskillsboost.google/paths/2294/course_templates/1235/labs/527176)

Note: this ipy-notebook was created step-by-spet according to the lab instructions (there is no a template file "sec_filings_analysis.ipynb" that is ready to use)

### Task 1. Prepare the environment in Vertex AI Workbench

In [ ]:
import os
import re
import requests
from datetime import datetime

from bs4 import BeautifulSoup, NavigableString

import vertexai
from vertexai.generative_models import (
    GenerativeModel,
    GenerationConfig,
    Tool,
    FunctionDeclaration,
    Part)

In [ ]:
#  initialize Vertex AI:
project_id = !gcloud config get project
project_id = project_id[0]

vertexai.init(project=project_id, location="us-central1")

### Task 2. Download SEC filings and parse text to analyze

In [ ]:
# IMPORTANT! You must set these variables with REAL VALUES for the correct headers to be in place to access the API
PARTNER_COMPANY = "Alphabet, Inc."
PARTNER_WEBSITE = "https://www.alphabet.com/"

headers = {"User-Agent": f"{PARTNER_COMPANY} +{PARTNER_WEBSITE}"}

In [ ]:
# Use the following code to name a local directory for storing filings and defining the following two functions:

#  - download_single_filing - It downloads a single SEC filing identified by a company's Central Index Key (CIK), the form type(an annual 10-K or quarterly 10-Q) and the date of filing.
#  - download_range_of_filings - It queries all the filings of a company to find filings within a date range of a particular form type and download them.


BASE_DIR = "filings" # Directory for storing raw filings

def download_single_filing(url, cik, form, date, file_extension):
    """
    This function downloads and saves a SEC filing from a specified URL.

    Args:
        cik (str): Central Index Key (CIK) for the company.
        form (str): The type of SEC filing (e.g., 10-K, 10-Q).
        date (str): The filing date in YYYY-MM-DD format.
        file_extension (str): File extension for saved file (usually 'txt' or 'zip').
    """

    # Define request headers to simulate a browser visit
    response = requests.get(url, headers=headers)

    if response.status_code == 200:

        # Make the directory accord
        dir_name = f"{BASE_DIR}/{cik}"
        os.makedirs(dir_name, exist_ok=True)
        file_path = f"{dir_name}/{form}_{date}.{file_extension}"

        with open(file_path, 'wb') as file:
            file.write(response.content)

        print(f"Downloaded {form} for CIK {cik} on {date} to {file_path}")

        return file_path

    else:
        print(f"Failed to download {form} for CIK {cik} on {date}. Status code: {response.status_code}")

        return None

def download_range_of_filings(cik, starting_year_and_quarter,
                            ending_year_and_quarter, include_10q = False):
    """
    Download filings from EDGAR for a given CIK and clean them up.

    Args:
        cik (str): Central Index Key (CIK)
        starting_year_and_quarter (str): Specified in the format "2023 Q1"
        ending_year_and_quarter (str): Specified in the format "2024 Q4"
        include_10q (bool): Whether to include 10-Qs in addition to 10-Ks
    """

    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    headers = {'User-Agent': 'Google Partner Learning Services Demo +https://partners.cloud.google.com/learn'}
    response = requests.get(url, headers=headers)

    forms_to_download = {"10-K", "10-Q"} if include_10q else {"10-K"}

    start_year, start_quarter = starting_year_and_quarter.split()
    end_year, end_quarter = ending_year_and_quarter.split()

    if response.status_code == 200:
        data = response.json()

        filing_paths = []
        for filing_date, form, accession_number in zip(data['filings']['recent']['filingDate'], data['filings']['recent']['form'], data['filings']['recent']['accessionNumber']):
            if (form in forms_to_download) and (int(start_year) <= datetime.strptime(filing_date, '%Y-%m-%d').year <= int(end_year)):
                file_path = download_single_filing(
                    f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_number.replace('-', '')}/{accession_number}.txt",
                    cik, form, filing_date, 'htm'
                )
                filing_paths.append (file_path)
        return filing_paths
    else:
        print("Error from call.")
        print(response)

In [ ]:
# Next, test the functions using the CIK of Alphabet, Google's parent company, which is 0001652044.

alphabet_cik = "0001652044"

download_range_of_filings(cik=alphabet_cik,
                starting_year_and_quarter="2024 Q1",
                ending_year_and_quarter="2024 Q4")

In [ ]:
# Instantiate a GenerativeModel using Gemini Pro version gemini-2.0-flash
model = GenerativeModel("gemini-2.0-flash",
                        generation_config=GenerationConfig(temperature=0),
                       )

In [ ]:
# To determine if you can send the entire document to Gemini to analyze at once, use your GenerativeModel's count_tokens method to see the number of tokens in the raw file.
downloaded_path = "filings/0001652044/10-K_2024-01-31.htm"

with open(downloaded_path, 'r') as f:
    filing_text = f.read()

response = model.count_tokens(filing_text)
print(response)

In [ ]:
'''
==> output:
total_tokens: 5798629
total_billable_characters: 13029439
prompt_tokens_details {
  modality: TEXT
  token_count: 5798629
}

(!) With a token count of 5,798,629, you can see that even 
with Gemini 2.0 Flash's large token window of 1,048,576 tokens,
these documents are too long to read in a single pass.

You could implement a Retrieval-Augmented Generation (RAG) framework
to query small chunks of these documents,
but here you are looking for a broader understanding of sections
as a whole rather than smaller chunks consisting of a few facts in the document.

Here, you don't mind passing a large number of tokens to Gemini,
as this will be an internal tool used by a relatively small number of analysts,
not a public tool handling thousands of queries.

(*)
SEC filings are required to adhere to a strict structure with named sections.
You can use this standard structure of the documents to read the text
between one section header and the next.
'''

In [ ]:
# Run the following code in your notebook to define a list of the sections(items) and functions to help retrieve all of the text from one item header until the next item header.

items = ['Business',
        'Risk Factors',
        'Unresolved Staff Comments',
        'Properties', 'Legal Proceedings',
        'Mine Safety Disclosures',
        'Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities',
        'Management’s Discussion and Analysis of Financial Condition and Results of Operations',
        'Quantitative and Qualitative Disclosures About Market Risk',
        'Financial Statements and Supplementary Data',
        'Changes in and Disagreements with Accountants on Accounting and Financial Disclosure',
        'Controls and Procedures',
        'Other Information',
        'Disclosure Regarding Foreign Jurisdictions that Prevent Inspections',
        'Directors, Executive Officers, and Corporate Governance',
        'Executive Compensation',
        'Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters',
        'Certain Relationships and Related Transactions, and Director Independence',
        'Principal Accountant Fees and Services',
        'Exhibit and Financial Statement Schedules',
        'Form 10-K Summary']

def find_div_id(soup, item_name: str):
    found_tag = soup.find('a', string=item_name)
    if found_tag:
        return found_tag["href"].strip('#')
    else:
        print(f"Couldn't find a matching tag for: {item_name}")
        return None

def get_text_between(soup, cur_name, end_name):
    cur = soup.find('div', id=find_div_id(soup, cur_name)).next_sibling
    end = soup.find('div', id=find_div_id(soup, end_name))
    while cur and cur != end:
        if isinstance(cur, NavigableString):
            text = cur.strip()
            if len(text):
                yield text
        cur = cur.next_element

def get_items_from_filings(item_names, filing_paths):

    print("Items of interest: " + ", ".join(item_names) + "\n")
    item_strings = {item: f"<{item}>\n" for item in item_names}

    for path in filing_paths:
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read()

        soup = BeautifulSoup(content, 'html.parser')

        for item in item_names:
            item_index = items.index(item)
            item_index = item_index if item_index < len(items) - 1 else 0
            item_output = ' '.join(text for text in get_text_between(soup, item, items[item_index + 1]))
            item_strings[item] += f"From {os.path.basename(path)}" + "\n" + item_output + "\n"

    return "\n\n".join(item_strings.values())    

In [ ]:
# Next, test the above functions by running the following command and save your notebook.

item_names = ["Management’s Discussion and Analysis of Financial Condition and Results of Operations"]

report = get_items_from_filings(item_names, [downloaded_path])

# Print the first 2,000 characters as an example.
print(report[0:2000] + "...")

In [ ]:
# Count the number of tokens in this section to see if it is under Gemini's token window of 1,048,576 tokens.
response = model.count_tokens(report)
print(response)

In [ ]:
'''
==> output:
total_tokens: 12688
total_billable_characters: 50620
prompt_tokens_details {
  modality: TEXT
  token_count: 12688
}
'''

### Task 3. Let Gemini look up companies' CIKs via Google Search

In [ ]:
# You were able to download Alphabet's annual report because you were provided
# its Central Index Key (CIK) number. Gemini already knows some CIKs for large public companies like Alphabet,
# but for some smaller companies, it may hallucinate and invent inaccurate CIKs.
# You can instead look up correct numbers by having Gemini use everyone's favorite well-known,
# public, up-to-date source of information: Google Search.

# Run the following command to see Gemini provide a CIK from its internal knowledge.
# The CIK of Summit Therapeutics is actually 0001599298, but without the ability 
# to look it up, Gemini provides some alternative numbers.

model.generate_content("What is Summit Therapeutics' CIK with the SEC?").text

'''
Output: "Summit Therapeutics' CIK is **0001522020**.
'''

In [ ]:
# Use Grounding with Google Search to provide Gemini with a tool to conduct Google searches.
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch, HttpOptions
from IPython.display import display, HTML
import os

project_id = os.environ['GOOGLE_CLOUD_PROJECT'] = 'qwiklabs-gcp-01-b76e37b08fba'
location = os.environ['GOOGLE_CLOUD_LOCATION'] = 'us-central1'
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

# Initialize the client, explicitly passing project and location for Vertex AI
client = genai.Client(project=project_id, location=location, http_options=HttpOptions(api_version="v1"))
model_id = "gemini-2.0-flash"

# Configure Google Search as a tool for grounding
search_tool = Tool(
    google_search=GoogleSearch()
)

# Helper function to lookup a company's SEC CIK
def lookup_cik(company_name: str) -> str:
    prompt = f"""
Find the Securities and Exchange Commission's Central Index Key (CIK)
for {company_name}.
Return only the 10-digit integer CIK including leading zeroes.
"""
    # Generate content with grounding via Google Search
    response = client.models.generate_content(
        model=model_id,
        contents=prompt,
        config=GenerateContentConfig(
            tools=[search_tool],
            response_modalities=["TEXT"],
        )
    )

    # Concatenate all text parts of the response
    cik = "".join(part.text for part in response.candidates[0].content.parts).strip()
    print(f"Lookup for CIK of {company_name} resulted in: {cik}\n")

    # Display the grounding metadata (search suggestions and sources)
    if hasattr(response.candidates[0], 'grounding_metadata'):
        display(
            HTML(
                response.candidates[0]
                .grounding_metadata.search_entry_point.rendered_content
            )
        )

    return cik

# Note: When you use Grounding with Google Search, you are required to display
# the corresponding Google Search Suggestions which help you understand 
# what Google search was conducted and allow you to investigate the results.

# The lookup_cik function below includes a display block that renders the provided HTML in this notebook.

In [ ]:
# Test the function and confirm you see the Google Search Suggestion results.
lookup_cik("Summit Therapeutics")

'''
Output:
Lookup for CIK of Summit Therapeutics resulted in: The Securities and Exchange Commission's (SEC) 
Central Index Key (CIK) for Summit Therapeutics is 0001599298.
'''

In [ ]:
# Create a FunctionDeclaration that will help Gemini know about function to lookup a CIK.
lookup_cik_fd = FunctionDeclaration(
    name="lookup_cik",
    description="Look up a company's CIK used for its SEC filings.",
    parameters={
        "type": "object",
        "properties": {
            "company_name": {
                "type": "string",
                "description": "The name of the company to look up."
            },
        },
        "required": [
            "company_name"
        ]
    },
)

In [ ]:
'''
Note:
    The search_tool loaded above cannot be combined with other Tools when passed to Gemini,
    so in order to create a Tool that combines it and other functions,
    you can create a dedicated model instance and invoke that via another function as you are doing here.
'''